## Material de Referencia
- SmileTokenizer https://deepchem.readthedocs.io/en/2.4.0/api_reference/tokenizers.html
- Transformer model to extract embedding and use it as input to another classifier. https://towardsdatascience.com/working-with-hugging-face-transformers-and-tf-2-0-89bf35e3555a

- Fine-tuning a pretrained model https://huggingface.co/transformers/training.html

In [1]:
%load_ext autoreload
%autoreload 2
import os.path
import pandas as pd
from sklearn.model_selection import train_test_split


from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout, Activation, BatchNormalization, Conv1D, MaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Flatten, Input, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras import layers as L

path = ''
if not os.path.isfile('001_Data_retrieve.ipynb'):
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/TP-FINAL/bioinformatics_final_project/'
    !pip install rdkit-pypi
    !pip install git+https://github.com/EBjerrum/molvecgen
    !pip install transformers
    !pip install deepchem

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    downloaded = drive.CreateFile({'id':'10fMCM9wnmjlyGwiyNzpfPROExiq8ukKk'})
    downloaded.GetContentFile('dataaug.py')
    downloaded = drive.CreateFile({'id':'1jPB1HDpGN5zFRuhqN6b9RLMuuUQy0UoB'})
    downloaded.GetContentFile('datagen.py')
    downloaded = drive.CreateFile({'id':'12FgtzBClWOeK6Kl3lEEV2YaTcXclHRzV'})
    downloaded.GetContentFile('smiles_tokenizer.py')

from datagen import DataGenerator
    

Mounted at /content/drive
     |████████████████████████████████| 18.6 MB 1.3 MB/s 
  Cloning https://github.com/EBjerrum/molvecgen to /tmp/pip-req-build-m22p810u
  Running command git clone -q https://github.com/EBjerrum/molvecgen /tmp/pip-req-build-m22p810u
  Created wheel for molvecgen: filename=molvecgen-0.1-py3-none-any.whl size=11386 sha256=58187f5e93eb643d50d0a3546489e88c60d19eb95fb34a6ff11800395c45c364
  Stored in directory: /tmp/pip-ephem-wheel-cache-l_zt86su/wheels/fe/f3/f6/b9c3409ce28e561f23bdc36bb487a3c9f181359e3ee4d22717
Successfully built molvecgen
     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 30.7 MB/s 
     |████████████████████████████████| 636 kB 63.3 MB/s 
     |████████████████████████████████| 895 kB 54.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 552 kB 5.1 MB/s 


## Carga de Datos

In [2]:
df = pd.read_csv(path+'data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')
max_len_idx = df['canonical_smiles'].apply(len).argmax()
min_len_idx = df['canonical_smiles'].apply(len).argmin()
max_sequence_len = len(df['canonical_smiles'].iloc[max_len_idx]) + 20

X = df['canonical_smiles'].values
y = df['pIC50'].values

## Split de Datos

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

if os.path.isfile('001_Data_retrieve.ipynb'):
  vocab_path = '/notebooks/TP-FINAL/bioinformatics_final_project/data'
else:
  vocab_path = path+'data'

dgen_train = DataGenerator(X_train, y_train, seq_length=max_sequence_len, batch_size=128, data_augmentation=True, smilesTokenizer=True, vocab_path=vocab_path )
dgen_test = DataGenerator(X_test, y_test, seq_length=max_sequence_len, batch_size=128, data_augmentation=False, smilesTokenizer=True, vocab_path=vocab_path)
token = dgen_train.get_token()

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.
Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


## MODELO TextCNN

In [ ]:
# Implementar modelo de TextCNN similar a la deepchem
def text_cnn_1d(sequence_length, vocab_size, embedding_size, filter_sizes, num_filters):
    # Recordar que estamos en la funcional model API y hay que entender que conectar con que.
    input_x = Input(shape=(sequence_length,), name='input_x')
                          # cant caracteres unicos, #long vect Emb,  #max vocabulario     
    embedding_layer = Embedding(vocab_size+1, embedding_size, input_length=sequence_length  )(input_x)
    pooled_outputs = []
    for filter_size in filter_sizes:
        conv1D = Conv1D( num_filters, filter_size, padding='valid')(embedding_layer) #sin activacion !!
        max_p = GlobalMaxPooling1D()(conv1D)
        pooled_outputs.append(max_p)
        
    h_pool = Concatenate(axis=1)(pooled_outputs)
    ##h_pool = Flatten()(h_pool)  #Flatten porque tengo MaxPool1D en vez de GlobalMaxPooling1D?
    h_pool = Dropout(0.1)(h_pool)
    dense = Dense(200, activation='relu')(h_pool)
    
    #dense = Dense(50, activation='relu')(dense)
    dense = Dense(1)(dense) # Salida
    model = Model(input_x, dense)
    return model

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

#Callbacks
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2)
mcp = ModelCheckpoint(path+'/models/best_model', save_best_only=True, save_format="h5")


In [ ]:
FILTER_SIZES = (3, 4, 5)
NUM_FILTERS = 128
vocab_size = token.vocab_size#len(smiles_dict)+1
embeddings_size = 128 # 2

In [ ]:
model = text_cnn_1d(max_sequence_len, vocab_size, embeddings_size, FILTER_SIZES, NUM_FILTERS)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=[R2])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_x (InputLayer)            [(None, 227)]        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 227, 128)     75776       input_x[0][0]                    
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 225, 128)     49280       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 224, 128)     65664       embedding_3[0][0]                
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=[R2])

In [ ]:
history = model.fit(dgen_train, epochs=2000, validation_data=dgen_test, callbacks=[earlystop])

/content/datagen.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_token = np.array([self.token.encode(smile) for smile in smilesBatch])


Epoch 1/2000
32/32 [==============================] - 45s 1s/step - loss: 30.9195 - R2: -10.0988 - val_loss: 24.1586 - val_R2: -7.3696
Epoch 2/2000
32/32 [==============================] - 45s 1s/step - loss: 16.6795 - R2: -4.9873 - val_loss: 8.7439 - val_R2: -2.0293
Epoch 3/2000
32/32 [==============================] - 45s 1s/step - loss: 4.4395 - R2: -0.5936 - val_loss: 2.6648 - val_R2: 0.0768
Epoch 4/2000
32/32 [==============================] - 45s 1s/step - loss: 2.5884 - R2: 0.0709 - val_loss: 2.6163 - val_R2: 0.0936
Epoch 5/2000
32/32 [==============================] - 45s 1s/step - loss: 2.5229 - R2: 0.0944 - val_loss: 2.5663 - val_R2: 0.1109
Epoch 6/2000
32/32 [==============================] - 46s 1s/step - loss: 2.4838 - R2: 0.1084 - val_loss: 2.5196 - val_R2: 0.1271
Epoch 7/2000
32/32 [==============================] - 46s 1s/step - loss: 2.4464 - R2: 0.1218 - val_loss: 2.4716 - val_R2: 0.1437
Epoch 8/2000
32/32 [==============================] - 45s 1s/step - loss: 2.4091 